In [63]:
import pandas as pd
import numpy as np
import joblib
import sklearn
# import ensemble
from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,mean_squared_error,balanced_accuracy_score,recall_score
import random
import lightgbm as lgb
import catboost as cbt

In [64]:
data = pd.read_parquet("Data/Training/Amex_Campus_Challenge_Train.parquet")

In [65]:
randomlist = random.sample(list(data.customer.unique()),int(0.2*len(data.customer.unique())))

In [66]:
testdata = data[data['customer'].isin(randomlist)]

In [67]:
data = data[~data['customer'].isin(randomlist)]

In [68]:
#select rows whose column value is not in a list
# traindata = data[~data['customer'].isin(randomlist)]
traindata = data

In [69]:
del data

In [70]:
def confusion_matrix_2col(df, col1, col2):
    return df.groupby([col1, col2]).size().unstack(fill_value=0)

confusion_matrix_2col(traindata, 'ind_recommended', 'activation')

activation,0,1
ind_recommended,,
0,8502584,48302
1,1228199,7812


In [71]:
tdata = traindata.drop(traindata[traindata['ind_recommended'] == 1].index)

In [72]:
# tdata = traindata.drop(traindata[traindata['activation'] == 0].sample(9661108).index)
tdata = tdata.drop(tdata[tdata['activation'] == 0].sample(8451108).index)
# tdata = tdata.drop(tdata[(tdata['activation'] == 1) & (tdata['ind_recommended'] == 0)].sample(35996).index)

In [73]:
X_train, X_test, y_train, y_test = train_test_split(tdata.drop(['customer','merchant','activation','ind_recommended'],axis=1), tdata['activation'], test_size=0.1, random_state=1)
act = lgb.LGBMClassifier(n_estimators=1500,learning_rate=0.01,objective='binary',metric='binary_logloss',random_state=1)

In [74]:
act.fit(X_train, y_train)
y_pred = act.predict(X_test)
print("mse : ",mean_squared_error(y_test, y_pred))

mse :  0.12778111846061335


In [75]:
print("Accuracy : ",accuracy_score(y_test, y_pred),"Balanced Accuracy :",balanced_accuracy_score(y_test, y_pred)," recall :",recall_score(y_test, y_pred))

Accuracy :  0.8722188815393866 Balanced Accuracy : 0.8727396274457062  recall : 0.8861313868613139


In [76]:
recomdata = traindata[(traindata['ind_recommended'] == 1) & (traindata['activation'] == 1)]

In [77]:
len(recomdata)

7812

In [78]:
recomdata['organicactivation'] = act.predict(recomdata.drop(['customer','merchant','activation','ind_recommended'],axis=1))

/var/folders/yy/95rx0nm12jqc32p5svcxxv7h0000gn/T/ipykernel_47813/355737933.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recomdata['organicactivation'] = act.predict(recomdata.drop(['customer','merchant','activation','ind_recommended'],axis=1))


In [79]:
recomdata['activation'] = (1-recomdata['organicactivation'])

/var/folders/yy/95rx0nm12jqc32p5svcxxv7h0000gn/T/ipykernel_47813/301614160.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recomdata['activation'] = (1-recomdata['organicactivation'])


In [80]:
X_train, X_test, y_train, y_test = train_test_split(recomdata.drop(['customer','merchant','activation','ind_recommended','organicactivation'],axis=1), recomdata['activation'], test_size=0.1, random_state=1)
rec = lgb.LGBMClassifier()

In [81]:
rec.fit(X_train, y_train)
y_pred = rec.predict(X_test)
print("mse : ",mean_squared_error(y_test, y_pred))

mse :  0.034526854219948847


In [82]:
print("Accuracy : ",accuracy_score(y_test, y_pred),"Balanced Accuracy :",balanced_accuracy_score(y_test, y_pred)," recall :",recall_score(y_test, y_pred))

Accuracy :  0.9654731457800512 Balanced Accuracy : 0.8575928134784554  recall : 0.7236842105263158


In [105]:
recomdata.activation.value_counts()

0    7043
1     769
Name: activation, dtype: int64

In [91]:
rec_list = rec.predict(testdata.drop(['ind_recommended','activation','customer','merchant'],axis=1))

In [96]:
act_list = act.predict_proba(testdata.drop(['ind_recommended','activation','customer','merchant'],axis=1))

In [100]:
act_list = [i[1] for i in act_list]

In [113]:
act_list = np.array(act_list)

In [114]:
testdata['predicted_score'] = (1-act_list)

In [115]:
evalu(testdata[['customer','predicted_score','merchant']],testdata[['customer','ind_recommended','activation','merchant']])

/var/folders/yy/95rx0nm12jqc32p5svcxxv7h0000gn/T/ipykernel_47813/245156590.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_input['predicted_score'] = df_input['predicted_score'].round(10)


Input Files are Correct
   ind_recommended  avg_30d_act
0                0     0.000414
1                1     0.000827
Incremental Activation Rate for Top 10 ranked Merchants(dataset level):  0.0004123


In [94]:
# testdata[['customer','predicted_score','merchant']].to_csv('d_pred.csv',index=False)

In [95]:
name='allfourregressionwcatboostimbalancefine'

In [40]:
import os
os.makedirs('submissions/'+name+'/models', exist_ok=True)

In [41]:
joblib.dump(rec, 'submissions/'+name+'/models/score.joblib')

['submissions/allfourregressionwcatboostimbalancefine/models/score.joblib']

In [42]:
testdata.predicted_score.mean()

1.9987477589695894

In [ ]:
#form1  (rec.round-0.5)*act

In [61]:
def incr_act_top10(input_df,pred_col,cm_key='customer',treated_col='ind_recommended',actual_col='activation'):
    
	#for correcting variable types
    input_df[[treated_col, actual_col, pred_col]] = input_df[[treated_col, actual_col, pred_col]].apply(pd.to_numeric, errors='coerce')
	
    input_df['rank_per_cm1'] = input_df.groupby(cm_key)[pred_col].rank(method='first', ascending=False)
    
    input_df = input_df.loc[input_df.rank_per_cm1 <= 10,:]
    
    agg_df = input_df.groupby(treated_col,as_index=False).agg({actual_col:'mean'})
    agg_df.columns = [treated_col,'avg_30d_act']
    
    print(agg_df)
    recommended_avg_30d_act = float(agg_df.loc[agg_df[treated_col]==1,'avg_30d_act'])
    not_recommended_avg_30d_act = float(agg_df.loc[agg_df[treated_col]==0,'avg_30d_act'])
    
    return (recommended_avg_30d_act-not_recommended_avg_30d_act)

In [60]:
def evalu(df_input, df_round): 
    # round off scores to 10 decimal points
    df_input['predicted_score'] = df_input['predicted_score'].round(10)


    # groupby customer, merchant and max score
    df_input = df_input.groupby(['customer', 'merchant'], as_index = False)['predicted_score'].agg('max')


    # merging predicted file and dependent variable file
    eval_data = pd.merge(df_round,df_input,on=['customer','merchant'],how='inner').drop_duplicates()
    # deleting the rows having null value in predicted_score
    eval_data = eval_data[~(eval_data['predicted_score'].isna())]


    if df_round.shape[0] != eval_data.shape[0]:
        return('Error: Rows are missing in the output file')
    else:
        print('Input Files are Correct')

    final_score = round(incr_act_top10(input_df=eval_data,pred_col='predicted_score',cm_key='customer',treated_col='ind_recommended',actual_col='activation'), 7)

    print('Incremental Activation Rate for Top 10 ranked Merchants(dataset level): ', final_score)